![cactus](http://cactuscode.org/global/images/cactuslogo.png)
# Tutorial 2: Reworking Symmetry Boundary Conditions

If you still have the Tutorial directory from the first tutorial, you do not need to run these commands. If you need to rebuild the toolkit from scratch, you can run this without needing anything from the previous tutorial.

In [ ]:
%cd ~/
%mkdir -p Tutorial
%cd ~/Tutorial
!curl -kLO https://raw.githubusercontent.com/gridaphobe/CRL/ET_2018_02/GetComponents
!chmod a+x ./GetComponents
!echo no|./GetComponents --parallel ~/PreSyncTutorial/thorns.th
%cd ~/Tutorial/CactusPre
!./simfactory/bin/sim setup-silent
%cd ~/Tutorial/CactusPre
!time ./simfactory/bin/sim build -j 2 --thornlist=~/Tutorial/CactusPre/thornlists/thorns.th

<h1>Using Symmetry BCs</h1>

In the first tutorial, we successfully updated a thorn to use PreSync. This demonstrated two different update types:
1. updating a thorn that doesn't include boundary routines
    1. add READ/WRITE declarations to scheduled functions
    2. place all old boundary scheduling into a legacy if statement
    3. add scheduling of BC selection routines in the PreSync_Selection group
    4. change DECLARE_CCTK_ARGUMENTS to the new macro and include the cctk_Arguments_Checked.h header file
2. Updating a boundary condition registered and applied using the Boundary thorn
    1. apply the changes for a standard thorn
    2. include the PreSync.h header file in the files with the boundary condition and registration functions

The second case has an important qualifier: it was specifically for boundary conditions which use Boundary to apply their boundary conditions. How do we change a boundary routine which doesn't use Boundary? In addition to this, there exists another less obvious case to consider. Symmetry boundary conditions (scheduled in BoundaryConditions group in Boundary) need to be applied to all variables, unlike physical boundary conditions. This means that PreSync needs to use a different system to apply them than the physical boundary conditions.

in this tutorial, we examine the symmetry boundary condition registered by the thorn ReflectionSymmetry thorn and how this thorn is changed to use PreSync. To start, we need to access ReflectionSymmetry without the PreSync changes. To do this, we will copy the master branch into the Tutorial directory so we may compare more easily inside this notebook.

In [ ]:
%cd ~/Tutorial/CactusPre/repos/cactusnumerical
!git checkout master
%cp -r ReflectionSymmetry ~/Tutorial/
!git checkout presync

Below, we use diff to see how the schedule.ccl changed for PreSync. The first difference is simply a renamed function, so we can ignore that change. The second is a little more involved. Originally, symmetry boundary conditions were applied by the Boundary thorn. Symmetry functions were registered with Symbase, and then the ApplyBC function was scheduled inside the BoundaryConditions group provided by the Boundary thorn. This scheduled function should only run if PreSync is not active, so it checks the value of use_psync. In addition, the name changes from ReflectionSymmetry_ApplyBC to Old_ReflectionSymmetry_ApplyBC. This is because the argument lists are different for the two functions, so they must be declared separately. We will see the details of this change later.

In [ ]:
!diff ~/Tutorial/CactusPre/repos/cactusnumerical/ReflectionSymmetry/schedule.ccl ~/Tutorial/ReflectionSymmetry/schedule.ccl

Next, we need to look at the interface.ccl. Below, we can see that a new function has been added named Boundary_RegisterSymmetryBC. This is a new function provided by Boundary2 to register symmetry boundary conditions with PreSync. The implementation is very similar to the Boundary_RegisterPhysicalBC provided by Boundary and Boundary2. Any thorn which provides symmetry boundary conditions must use this function to properly interface with PreSync. Also, most symmetry functions use Boundary_SelectedGVs. To properly implement the new function later, we must change the REQUIRES to USES. This is because it is only provided by Boundary, not Boundary2, and REQUIRES will demand that it is provided even though it is not used.

In [ ]:
!diff ~/Tutorial/CactusPre/repos/cactusnumerical/ReflectionSymmetry/interface.ccl ~/Tutorial/ReflectionSymmetry/interface.ccl

We can now move to the source code. Several files use the DECLARE_CCTK_ARGUMENTS macro, so these are changed to the new macro. However, this is a simple change, so we won't discuss it further. The next file with meaningful changes is register.c, which contains the function that registers the symmetry boundary condition with SymBase. The first difference is just a new name for the function, which we saw earlier in the schedule.ccl. the second change swaps out the macros. The last change adds several new lines of code. We see that it is only activated if PreSync is turned on, in which case it calls the Boundary_RegisterSymmetryBC function. We could also have added a completely new function to register with PreSync, but we chose to just have a single registration function.

In [ ]:
!diff ~/Tutorial/CactusPre/repos/cactusnumerical/ReflectionSymmetry/src/register.c ~/Tutorial/ReflectionSymmetry/src/register.c

We only have two more files with changes! The first is the header file Symmetry.h. For PreSync, we included PreSync.h and also included a prototype for the ApplyBC function. The prototyping is so we can pass that function in the Boundary_RegisterSymmetryBC call. In this case, we also removed some unnecessary prototyping lines, but they could have been left with no effect.

In [ ]:
!diff ~/Tutorial/CactusPre/repos/cactusnumerical/ReflectionSymmetry/src/reflection.h ~/Tutorial/ReflectionSymmetry/src/reflection.h

The last file,  has the most significant changes, as it contains the symmetry function ReflectionSymmetry_Apply. Below, we show the function declarations, including their argument lists.

Old function:
<div style='border-style: solid; border-color: black; padding: 5px;'>
<pre>
void ReflectionSymmetry_Apply(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
</pre>
</div>


New function:
<div style='border-style: solid; border-color: black; padding: 5px;'>
<pre>

void ReflectionSymmetry_Apply(const cGH *GH, CCTK_INT nvars, CCTK_INT *indices,
                 CCTK_INT *faces, CCTK_INT *boundary_widths,
                 CCTK_INT *table_handle) {
</pre>
</div>


The old declaration takes CCTK_ARGUMENTS as its only arguments, as it is a scheduled function. The new function instead takes several arguments. These are identical to the arguments passed to physical boundary functions. This change removes several function calls within symmetry boundary condition functions, as we'll soon see.

Let us examine the two functions in their entirety to see what changed inside the functions. The old function declares the variables that are arguments in the new function, and it then uses the Boundary_SelectedGVs function provided by the Boundary thorn to fill in the correct data. Then there's a loop over the variables that calls several internal routines to apply the boundary condition. Finally, the allocated variables are freed.

<div style='border-style: solid; border-color: black; padding: 5px;'>
<pre>
void ReflectionSymmetry_Apply (CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  
  int nvars;
  CCTK_INT * restrict indices;
  CCTK_INT * restrict faces;
  CCTK_INT * restrict widths;
  CCTK_INT * restrict tables;
  int vi;
  int dim;
  int * restrict stencil;
  int i;
  int istat;
  int ierr;
  
  if (! cctkGH) {
    CCTK_WARN (0, "Argument cctkGH is NULL");
  }
  
  nvars = Boundary_SelectedGVs (cctkGH, 0, NULL, NULL, NULL, NULL, NULL);
  if (nvars < 0) {
    CCTK_WARN (0, "Internal error in Boundary_SelectedGVs");
  }
  
  if (nvars == 0) {
    /* Nothing to do */
    return;
  }
  
  indices = malloc (nvars * sizeof *indices);
  if (! indices) {
    CCTK_WARN (0, "Out of memory");
  }
  faces = malloc (nvars * sizeof *faces);
  if (! faces) {
    CCTK_WARN (0, "Out of memory");
  }
  widths = malloc (nvars * sizeof *widths);
  if (! widths) {
    CCTK_WARN (0, "Out of memory");
  }
  tables = malloc (nvars * sizeof *tables);
  if (! tables) {
    CCTK_WARN (0, "Out of memory");
  }
  
  istat =  Boundary_SelectedGVs
    (cctkGH, nvars, indices, faces, widths, tables, 0);
  if (istat != nvars) {
    CCTK_WARN (0, "Internal error in Boundary_SelectedGVs");
  }
  
  for (i=0; i<nvars; ++i) {
    vi = indices[i];
    if (vi < 0 || vi >= CCTK_NumVars()) {
      CCTK_WARN (0, "Illegal variable index");
    }
    
    if (widths[i] < 0) {
      CCTK_WARN (0, "Illegal boundary width");
    }
    
    dim = CCTK_GroupDimFromVarI (vi);
    if (dim < 0) {
      CCTK_WARN (0, "Illegal dimension");
    }
    
    stencil = malloc (dim * sizeof *stencil);
    if (! stencil) {
      CCTK_WARN (0, "Out of memory");
    }
    ierr = CCTK_GroupnghostzonesVI (cctkGH, dim, stencil, vi);
    if (ierr) {
      CCTK_WARN (0, "Internal error in CCTK_GroupnghostzonesVI");
    }
    
    CheckBoundaryParameters (cctkGH, vi, stencil);
    
    ierr = BndReflectVI (cctkGH, vi);
    if (ierr) {
      CCTK_WARN (0, "Internal error in BndReflectVI");
    }
    
    free (stencil);
  }
  
  free (indices);
  free (faces);
  free (widths);
  free (tables);
}
</pre>
</div>

In contrast, here is the new version of the function. Since the variables that were retrieved from Boundary are passed in directly, we no longer need the majority of the code. In the case of ReflectionSymmetry, these variables are not actually used in the remainder of the code. This is not necessarily true for all symmetry functions, so it still needs to use the same argument list as all other symmetry boundary conditions. Many of these error-checking lines could likely be removed as well. For instance, the first check is unnecessary, as the passed variable index is guarranteed to be a legitimate value. 
<div style='border-style: solid; border-color: black; padding: 5px;'>
<pre>
void ReflectionSymmetry_Apply(const cGH *cctkGH, CCTK_INT nvars, CCTK_INT *indices,
                 CCTK_INT *faces, CCTK_INT *widths,
                 CCTK_INT *tables)
{
  
  int vi;
  int dim;
  int * restrict stencil;
  int i;
  int istat;
  int ierr;
  
  if (! cctkGH) {
    CCTK_WARN (0, "Argument cctkGH is NULL");
  }
  
  for (i=0; i<nvars; ++i) {
    vi = indices[i];
    if (vi < 0 || vi >= CCTK_NumVars()) {
      CCTK_WARN (0, "Illegal variable index");
    }
    
    if (widths[i] < 0) {
      CCTK_WARN (0, "Illegal boundary width");
    }
    
    dim = CCTK_GroupDimFromVarI (vi);
    if (dim < 0) {
      CCTK_WARN (0, "Illegal dimension");
    }
    
    stencil = malloc (dim * sizeof *stencil);
    if (! stencil) {
      CCTK_WARN (0, "Out of memory");
    }
    ierr = CCTK_GroupnghostzonesVI (cctkGH, dim, stencil, vi);
    if (ierr) {
      CCTK_WARN (0, "Internal error in CCTK_GroupnghostzonesVI");
    }
    
    CheckBoundaryParameters (cctkGH, vi, stencil);
    
    ierr = BndReflectVI (cctkGH, vi);
    if (ierr) {
      CCTK_WARN (0, "Internal error in BndReflectVI");
    }
    
    free (stencil);
  }
}
</pre>
</div>

Below, we see the new functions in their entirety. To update a symmetry boundary condition for PreSync, we first copy the original code and name it Old_FunctionName, to match the legacy scheduling we added earlier. Then, we remove the lines that allocate and fill the variables indices, faces, widths, and tables at the beginning of the function. We also remove the free() operations at the end.
<div style='border-style: solid; border-color: black; padding: 5px;'>
<pre>
void ReflectionSymmetry_Apply(const cGH *cctkGH, CCTK_INT nvars, CCTK_INT *indices,
                 CCTK_INT *faces, CCTK_INT *widths,
                 CCTK_INT *tables)
{
  
  int vi;
  int dim;
  int * restrict stencil;
  int i;
  int istat;
  int ierr;
  
  if (! cctkGH) {
    CCTK_WARN (0, "Argument cctkGH is NULL");
  }
  
  for (i=0; i<nvars; ++i) {
    vi = indices[i];
    if (vi < 0 || vi >= CCTK_NumVars()) {
      CCTK_WARN (0, "Illegal variable index");
    }
    
    if (widths[i] < 0) {
      CCTK_WARN (0, "Illegal boundary width");
    }
    
    dim = CCTK_GroupDimFromVarI (vi);
    if (dim < 0) {
      CCTK_WARN (0, "Illegal dimension");
    }
    
    stencil = malloc (dim * sizeof *stencil);
    if (! stencil) {
      CCTK_WARN (0, "Out of memory");
    }
    ierr = CCTK_GroupnghostzonesVI (cctkGH, dim, stencil, vi);
    if (ierr) {
      CCTK_WARN (0, "Internal error in CCTK_GroupnghostzonesVI");
    }
    
    CheckBoundaryParameters (cctkGH, vi, stencil);
    
    ierr = BndReflectVI (cctkGH, vi);
    if (ierr) {
      CCTK_WARN (0, "Internal error in BndReflectVI");
    }
    
    free (stencil);
  }
}

/*************************************
   This function exists to maintain
   PreSync backward compatibility.
 *************************************/
void
Old_ReflectionSymmetry_Apply (CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  
  int nvars;
  CCTK_INT * restrict indices;
  CCTK_INT * restrict faces;
  CCTK_INT * restrict widths;
  CCTK_INT * restrict tables;
  int vi;
  int dim;
  int * restrict stencil;
  int i;
  int istat;
  int ierr;
  
  if (! cctkGH) {
    CCTK_WARN (0, "Argument cctkGH is NULL");
  }
  
  nvars = Boundary_SelectedGVs (cctkGH, 0, NULL, NULL, NULL, NULL, NULL);
  if (nvars < 0) {
    CCTK_WARN (0, "Internal error in Boundary_SelectedGVs");
  }
  
  if (nvars == 0) {
    /* Nothing to do */
    return;
  }
  
  indices = malloc (nvars * sizeof *indices);
  if (! indices) {
    CCTK_WARN (0, "Out of memory");
  }
  faces = malloc (nvars * sizeof *faces);
  if (! faces) {
    CCTK_WARN (0, "Out of memory");
  }
  widths = malloc (nvars * sizeof *widths);
  if (! widths) {
    CCTK_WARN (0, "Out of memory");
  }
  tables = malloc (nvars * sizeof *tables);
  if (! tables) {
    CCTK_WARN (0, "Out of memory");
  }
  
  istat =  Boundary_SelectedGVs
    (cctkGH, nvars, indices, faces, widths, tables, 0);
  if (istat != nvars) {
    CCTK_WARN (0, "Internal error in Boundary_SelectedGVs");
  }
  
  for (i=0; i<nvars; ++i) {
    vi = indices[i];
    if (vi < 0 || vi >= CCTK_NumVars()) {
      CCTK_WARN (0, "Illegal variable index");
    }
    
    if (widths[i] < 0) {
      CCTK_WARN (0, "Illegal boundary width");
    }
    
    dim = CCTK_GroupDimFromVarI (vi);
    if (dim < 0) {
      CCTK_WARN (0, "Illegal dimension");
    }
    
    stencil = malloc (dim * sizeof *stencil);
    if (! stencil) {
      CCTK_WARN (0, "Out of memory");
    }
    ierr = CCTK_GroupnghostzonesVI (cctkGH, dim, stencil, vi);
    if (ierr) {
      CCTK_WARN (0, "Internal error in CCTK_GroupnghostzonesVI");
    }
    
    CheckBoundaryParameters (cctkGH, vi, stencil);
    
    ierr = BndReflectVI (cctkGH, vi);
    if (ierr) {
      CCTK_WARN (0, "Internal error in BndReflectVI");
    }
    
    free (stencil);
  }
  
  free (indices);
  free (faces);
  free (widths);
  free (tables);
}
</pre>
</div>

<h1>Testing PreSync Changes</h1>

Of course, up to now we have just been looking at the thorn differences. Let's run some tests to see how these changes affect things!

In PresyncWave, there are two tests - oldreflection and newreflection. These are similar to the oldsync and presync tests we made in the first tutorial. These are those tests, but the ReflectionSymmetry has been activated, and some parameters from ReflectionSymmetry have been given values.

To begin, we will use ReflectionSymmetry from the master branch to see how the tests fare before these changes.

In [ ]:
%cd ~/Tutorial/CactusPre
%cp -r ../ReflectionSymmetry/* repos/cactusnumerical/ReflectionSymmetry/
!time ./simfactory/bin/sim build -j 2 --thornlist=~/Tutorial/CactusPre/thornlists/thorns.th
!./simfactory/bin/sim create-run reflectiontest --testsuite --procs 2 --num-threads 1 --select-tests PresyncWave/PresyncWave

We see that the oldreflection test passes when we include the old ReflectionSymmetry thorn. From this, we can see that PreSync works with old thorns so long as the parameter file is also old. However, newreflection fails, which is expected. After all, this version of ReflectionSymmetry doesn't have any of our changes!

Let's revert this to the presync version of ReflectionSymmetry and try it again.

In [ ]:
%cd ~/Tutorial/CactusPre/repos/cactusnumerical
!git checkout ReflectionSymmetry
%cd ~/Tutorial/CactusPre
!time ./simfactory/bin/sim build -j 2 --thornlist=~/Tutorial/CactusPre/thornlists/thorns.th
!./simfactory/bin/sim create-run reflectiontest2 --testsuite --procs 2 --num-threads 1 --select-tests PresyncWave/PresyncWave

As expected, all the tests pass with the presync branch of ReflectionSymmetry. Now you've seen how to change a symmetry boundary condition routine to use PreSync!

<table><tr><td>This work sponsored by NSF grants <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1550551"> OAC 1550551</a> </td><td><img src="https://www.nsf.gov/awardsearch/images/common/nsf_logo_bottom.png"></tr></table>